In [1]:
import os
os.chdir('/home/piper/work')

import numpy as np
import megatron

In [2]:
# lazy
G = megatron.Graph(cache_path='megatron/cache')
X = megatron.FeatureSet(G, 'X', 10)
Y = megatron.FeatureSet(G, 'Y', 10)

Z = megatron.Transformation(megatron.transforms.time_series, window_size=5)(X)

data = {'X': np.ones((50, 10)),
        'Y': np.ones((50, 10))}

result = G.run(Z, data, cache=False)
print(result.shape)

(45, 5, 10)


In [3]:
start = G.run(Z, data)

Loading node number 1 in path from cache


In [4]:
# eager
data = {'X': np.random.random((50, 10)),
        'Y': np.random.random((50, 10))}

G = megatron.Graph()
X = megatron.FeatureSet(G, 'X', 10)(data['X'])
Y = megatron.FeatureSet(G, 'Y', 10)(data['Y'])
Z = megatron.Transformation(megatron.transforms.time_series, window_size=5)(X)
print(X.output.shape)
print(Z.output.shape)

(50, 10)
(45, 5, 10)


In [16]:
np.savez_compressed('test.npz', arr=start)

In [25]:
start2 = np.load('test.npz')['arr']